In [ ]:
import os
import time
import math
from functools import partial
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import sklearn.metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
base_path = "./data"
data_path = "Data.xlsx"

In [ ]:
data = pd.read_excel(os.path.join(base_path, data_path))
data

In [ ]:
del data["Article Number"]
data

In [ ]:
data.dtypes

In [ ]:
data = data.dropna()
data

In [ ]:
data.describe().transpose()[['mean', 'std']]

In [ ]:
def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)



# load package
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator,FormatStrFormatter,MaxNLocator
import matplotlib.font_manager as font_manager


mpl.rcParams['pdf.fonttype'] = 3
mpl.rcParams['ps.fonttype'] = 3
mpl.rcParams['font.family'] = 'Arial'


mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['axes.labelweight'] = "bold"


mpl.rcParams['xtick.labelsize'] = 8
mpl.rcParams['ytick.labelsize'] = 8


mpl.rcParams['legend.fontsize'] = 7


text_font = {'fontname':'Arial', 'size':'7'}

In [ ]:
plt.figure(figsize=(12, 9))
for counter, column in enumerate(data.columns[:12]):
    plt.subplot(3,4,counter+1)
    sns.kdeplot(data=data, x=column, multiple="stack")
    

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.6, 
                    hspace=0.35)
plt.savefig("distributions.pdf")

In [ ]:
data_ = data.to_numpy().astype(np.float32)
data_ = shuffle(data_, random_state=42)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def lr_scheduler(epoch, warmup_epochs=50, decay_epochs=450, initial_lr=1e-6, base_lr=5e-3, min_lr=5e-5):
    if epoch <= warmup_epochs:
        pct = epoch / warmup_epochs
        return ((base_lr - initial_lr) * pct) + initial_lr

    if epoch > warmup_epochs and epoch < warmup_epochs+decay_epochs:
        pct = 1 - ((epoch - warmup_epochs) / decay_epochs)
        return ((base_lr - min_lr) * pct) + min_lr

    return min_lr

In [ ]:
class BestModelWeights(tf.keras.callbacks.Callback):
    def __init__(self, metric="val_loss"):
        super(BestModelWeights, self).__init__()
        self.metric = metric

    def on_train_begin(self, logs=None):
        self.best_val_loss = np.inf
        self.best_epoch = 0
        self.model_best_weights = None
        
    def on_epoch_end(self, epoch, logs=None):
        if self.best_val_loss >= logs[self.metric]:
            self.model_best_weights = self.model.get_weights()
            self.best_val_loss = logs[self.metric]
            self.best_epoch = epoch

    def on_train_end(self, logs=None):
        self.model.set_weights(self.model_best_weights)
        print(f"Best weights is set, Best Epoch was : {self.best_epoch}")

In [ ]:
class ShowLoss(tf.keras.callbacks.Callback):
    def __init__(self, step_show):
        super(ShowLoss, self).__init__()
        self.step_show = step_show
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.step_show == 0:
            print(f"Train epoch : {epoch}")
            #print(f"Train [ Loss {round(logs['loss'], 4)}, MAE {round(logs['mae'], 4)}, Sensivity {round(logs['sensivity'], 4)}, Test [ Loss {round(logs['val_loss'], 4)}, MAE {round(logs['val_mae'], 4)}, Sensivity {round(logs['val_sensivity'], 4)}]")

In [ ]:
# Deep Neural Network
def create_model(in_features, o_features, n_h_layers=4, drop_rate=0.25, h_units=64):
    inputs = tf.keras.layers.Input(shape=(in_features,))
    x = inputs

    for _ in range(n_h_layers):
        x = tf.keras.layers.Dense(h_units, activation="relu")(x)
        x = tf.keras.layers.Dropout(drop_rate)(x)

    x = tf.keras.layers.Dense(o_features)(x)
    x = tf.keras.layers.ReLU(max_value=100.0)(x)
    outputs = tf.keras.layers.Lambda(lambda x: tf.squeeze(x))(x)

    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
# Linear Regression
def create_model(in_features, o_features):
    inputs = tf.keras.layers.Input(shape=(in_features,))
    x = inputs

    x = tf.keras.layers.Dense(o_features)(x)
    x = tf.keras.layers.ReLU(max_value=100.0)(x)
    outputs = tf.keras.layers.Lambda(lambda x: tf.squeeze(x))(x)

    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
# Polynomial Regression
def create_model(in_features, o_features, degree=10):
    inputs = tf.keras.layers.Input(shape=(in_features,))

    SumDegree = []
    for counter in range(2, degree+1):
        x = tf.keras.layers.Lambda(lambda x: tf.pow(x, counter))(inputs)
        SumDegree.append(tf.keras.layers.Dense(o_features, use_bias=False)(x))
    SumDegree.append(tf.keras.layers.Dense(o_features, use_bias=True)(inputs))
    
    outputs = tf.keras.layers.Add()(SumDegree)
    x = tf.keras.layers.ReLU(max_value=100.0)(x)
    outputs = tf.keras.layers.Lambda(lambda x: tf.squeeze(x))(outputs)

    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
def sensitivity(y_true, y_pred, pred_main_model=None):
    if pred_main_model is None:
        pred_main_model = y_true

    try:
        difference = 100.0 * tf.abs((pred_main_model - y_pred) / (pred_main_model + 1e-8))
    except:
        difference = 100.0 * tf.abs((y_true - y_pred) / (y_true + 1e-8))
        
    return tf.reduce_mean(difference, axis=-1)  # Note the `axis=-1`

r2score = tfa.metrics.RSquare()
rsme = tf.keras.metrics.RootMeanSquaredError(name='RMSE')
mae = tf.keras.metrics.MeanAbsoluteError(name='MAE')

epsilon = 5.0
def MAEM(y_true, y_pred):
    difference = tf.abs(y_true - y_pred) - epsilon
    return tf.nn.relu(difference)

In [ ]:
def model_train(model, train_data, test_data, initial_weights_path, epoch, pred_main_model=None):
    model.load_weights(initial_weights_path)

    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)
    #optimizer = tfa.optimizers.AdamW(weight_decay=1e-4, learning_rate=2e-4)

    sensitivity_ = partial(sensitivity, pred_main_model=pred_main_model)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredError(), #MAEM # tf.keras.losses.MeanAbsoluteError()
                  metrics=[sensitivity_, r2score, rsme, mae])
    
    callbacks = [tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0),
                 BestModelWeights(),
                 ShowLoss(100)]

    history = model.fit(train_data[0], train_data[1],
                        epochs=epoch,
                        batch_size=len(train_data[0]),
                        validation_data=test_data,
                        callbacks=callbacks,
                        verbose=0).history
    
    #best_index = np.argmin(history['val_loss'])
    best_index = np.argmin(history['val_partial'])

    return np.array([history['MAE'][best_index],
                     history['partial'][best_index],
                     history['RMSE'][best_index],
                     history['r_square'][best_index],
                     history['val_MAE'][best_index],
                     history['val_partial'][best_index],
                     history['val_RMSE'][best_index],
                     history['val_r_square'][best_index]])

In [ ]:
!rm -rf Result

In [ ]:
os.mkdir("Result")
os.mkdir("Result/Seperated")

In [ ]:
EPOCH = 500
n_splits = 5
n_repeats = 3
initial_weights = "./Result/initial_weights.h5"

kf = KFold(n_splits=n_splits)
X, Y = data_[:, :11], data_[:, 11:]

model = create_model(X.shape[-1], 1)
model.save_weights(initial_weights)


Result_No_Dropping = np.zeros((n_splits, 8, n_repeats))
Results_Dropping_EE = np.zeros((n_splits, X.shape[-1], 8, n_repeats))
for counter, (train_index, test_index) in enumerate(kf.split(X)):
    print(70 * "*", f"Fold {counter + 1} Started!", 70 * "*")
    
    train_x, train_y = X[train_index], Y[train_index]
    test_x, test_y = X[test_index], Y[test_index]
    

    ############################ With target normalization ###############################
    scaler_x = MinMaxScaler()
    
    scaler_x.fit(X)
    train_x = scaler_x.transform(train_x)
    test_x = scaler_x.transform(test_x)
    #######################################################################################



    train_y_EE = train_y[:, 0]
    test_y_EE = test_y[:, 0]


    pred_main_model = 0
    for counter1 in range(n_repeats):
        model = create_model(train_x.shape[-1], 1)
        best_result_metrics = model_train(model, (train_x, train_y_EE), (test_x, test_y_EE), initial_weights, EPOCH)
        pred_main_model += model(test_x)

        Result_No_Dropping[counter, :, counter1] = best_result_metrics
    pred_main_model = pred_main_model / n_repeats


    print(f"Fold {counter + 1} without dropping : Train [ MAE={round(Result_No_Dropping[counter,0,:].mean(), 4)}, Sensivity={round(Result_No_Dropping[counter,1,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,2,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,3,:].mean(), 4)} ], Test [ MAE={round(Result_No_Dropping[counter,4,:].mean(), 4)}, Sensivity={round(Result_No_Dropping[counter,5,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,6,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,7,:].mean(), 4)} ]")
    print(158 * "*")

    for counter1 in range(X.shape[1]):
        start = time.time()
        for counter2 in range(n_repeats):
            croped_train_x, croped_test_x = train_x.copy(), test_x.copy()

            croped_train_x[:, counter1] = -1
            croped_test_x[:, counter1] = -1
            
            
            model = create_model(croped_train_x.shape[-1], 1)
            best_result_metrics = model_train(model, (croped_train_x, train_y_EE), (croped_test_x, test_y_EE), initial_weights, EPOCH, pred_main_model)

            Results_Dropping_EE[counter, counter1, :, counter2] = best_result_metrics

        
        print(f"Fold  {counter + 1},  Feature {counter1 + 1} : Train [ MAE={round(Results_Dropping_EE[counter,counter1,0,:].mean(), 4)}, Sensivity={round(Results_Dropping_EE[counter,counter1,1,:].mean(), 4)}, RMSE={round(Results_Dropping_EE[counter,counter1,2,:].mean(), 4)}, R2={round(Results_Dropping_EE[counter,counter1,3,:].mean(), 4)} ], Test [ MAE={round(Results_Dropping_EE[counter,counter1,4,:].mean(), 4)}, Sensivity={round(Results_Dropping_EE[counter,counter1,5,:].mean(), 4)}, RMSE={round(Results_Dropping_EE[counter,counter1,6,:].mean(), 4)}, R2={round(Results_Dropping_EE[counter,counter1,7,:].mean(), 4)} ]")
        print(f"Time : {round(time.time() - start, 4)}")
        print(158 * "*")

In [ ]:
with open('./Result/Seperated/Results_Dropping.npy', 'wb') as f:
     np.save(f, Results_Dropping_EE)     

with open('./Result/Seperated/Result_No_Dropping.npy', 'wb') as f:
     np.save(f, Result_No_Dropping)

In [ ]:
Results_Dropping_EE.mean(axis=0).mean(axis=-1).shape

In [ ]:
import matplotlib.pyplot as plt


plt.plot(Results_Dropping_EE[:,:,5,:].mean(axis=0).mean(axis=-1))
plt.plot([Result_No_Dropping.mean(axis=0).mean(axis=-1)[5]] * 11)
plt.xticks(range(11), list(data.columns)[:11], rotation=90)

plt.show()

In [ ]:
!zip -r metrics.zip /content/Result

In [ ]:
predicted = model.predict(x_test)

In [ ]:
plt.scatter(y_test[:,0], predicted[:,0])
plt.plot(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01), "r")
plt.xlabel("Y True")
plt.ylabel("Y Predicted")
plt.title("Particle Size")


plt.text(0.3, 0.8, r'$R^2 = ${:.4f}'.format(r2_score(y_test[:,0], predicted[:,0])), fontsize=14)

In [ ]:
plt.scatter(y_test[:,1], predicted[:,1])
plt.plot(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01), "r")
plt.xlabel("Y True")
plt.ylabel("Y Predicted")
plt.title("EE %")

plt.text(0.3, 0.8, r'$R^2 = ${:.4f}'.format(r2_score(y_test[:,1], predicted[:,1])), fontsize=14)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def lr_scheduler(epoch, warmup_epochs=100, decay_epochs=900, initial_lr=1e-6, base_lr=5e-3, min_lr=5e-5):
    if epoch <= warmup_epochs:
        pct = epoch / warmup_epochs
        return ((base_lr - initial_lr) * pct) + initial_lr

    if epoch > warmup_epochs and epoch < warmup_epochs+decay_epochs:
        pct = 1 - ((epoch - warmup_epochs) / decay_epochs)
        return ((base_lr - min_lr) * pct) + min_lr

    return min_lr

In [ ]:
class BestModelWeights(tf.keras.callbacks.Callback):
    def __init__(self, metric="val_loss"):
        super(BestModelWeights, self).__init__()
        self.metric = metric

    def on_train_begin(self, logs=None):
        self.best_val_loss = np.inf
        self.best_epoch = 0
        self.model_best_weights = None
        
    def on_epoch_end(self, epoch, logs=None):
        if self.best_val_loss >= logs[self.metric]:
            self.model_best_weights = self.model.get_weights()
            self.best_val_loss = logs[self.metric]
            self.best_epoch = epoch

    def on_train_end(self, logs=None):
        self.model.set_weights(self.model_best_weights)
        print(f"Best weights is set, Best Epoch was : {self.best_epoch}\n")

In [ ]:
from tqdm.notebook import tqdm


class ShowProgress(tf.keras.callbacks.Callback):
    def __init__(self, epochs, step_show=50):
        super(ShowProgress, self).__init__()
        self.epochs = epochs
        self.step_show = step_show

    def on_train_begin(self, logs=None):
        self.pbar = tqdm(range(self.epochs))

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.step_show == 0:
            self.pbar.set_description(f"Epoch : {epoch + 1} / {self.epochs}, Train Loss : {round(logs['loss'], 4)}, Valid Loss : {round(logs['val_loss'], 4)}")
            self.pbar.update(self.step_show)

In [ ]:
# Deep Neural Network
def create_model(in_features, o_features, n_h_layers=4, drop_rate=0.25, h_units=64):
    inputs = tf.keras.layers.Input(shape=(in_features,))
    x = inputs

    for _ in range(n_h_layers):
        x = tf.keras.layers.Dense(h_units, activation="relu")(x)
        x = tf.keras.layers.Dropout(drop_rate)(x)

    x = tf.keras.layers.Dense(o_features)(x)
    outputs = tf.keras.layers.ReLU(max_value=100.0)(x)

    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
# Linear Regression
def create_model(in_features, o_features):
    inputs = tf.keras.layers.Input(shape=(in_features,))
    x = inputs

    x = tf.keras.layers.Dense(o_features)(x)
    outputs = tf.keras.layers.ReLU(max_value=100.0)(x)

    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
# Polynomial Regression
def create_model(in_features, o_features, degree=10):
    inputs = tf.keras.layers.Input(shape=(in_features,))

    SumDegree = []
    for counter in range(2, degree+1):
        x = tf.keras.layers.Lambda(lambda x: tf.pow(x, counter))(inputs)
        SumDegree.append(tf.keras.layers.Dense(o_features, use_bias=False)(x))
    SumDegree.append(tf.keras.layers.Dense(o_features, use_bias=True)(inputs))
    
    outputs = tf.keras.layers.Add()(SumDegree)
    outputs = tf.keras.layers.ReLU(max_value=100.0)(outputs)


    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
from typing import Tuple
from tensorflow_addons.utils.types import AcceptableDTypes
from tensorflow.python.ops import weights_broadcast_ops
from tensorflow.keras import backend as K




class RSquare(tf.keras.metrics.Metric):
    def __init__(
        self,
        uncertainty: bool = False,
        name: str = "r_square",
        dtype: AcceptableDTypes = None,
        y_shape: Tuple[int, ...] = (),
        multioutput: str = "uniform_average",
        num_regressors: tf.int32 = 0,
        **kwargs,
    ):
        super(RSquare, self).__init__(name=name, dtype=dtype, **kwargs)
        self.uncertainty = uncertainty
        self.y_shape = y_shape

        self.multioutput = multioutput
        self.num_regressors = num_regressors
        self.squared_sum = self.add_weight(
            name="squared_sum", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.sum = self.add_weight(
            name="sum", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.res = self.add_weight(
            name="residual", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.count = self.add_weight(
            name="count", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.num_samples = self.add_weight(name="num_samples", dtype=tf.int32)

    def update_state(self, y_true, y_pred, sample_weight=None) -> None:
        y_true = tf.cast(y_true, dtype=self._dtype)
        y_pred = tf.cast(y_pred, dtype=self._dtype)

        if self.uncertainty:
            y_pred = y_pred[:, ::2]


        if sample_weight is None:
            sample_weight = 1
        sample_weight = tf.cast(sample_weight, dtype=self._dtype)
        sample_weight = weights_broadcast_ops.broadcast_weights(
            weights=sample_weight, values=y_true
        )

        weighted_y_true = y_true * sample_weight
        self.sum.assign_add(tf.reduce_sum(weighted_y_true, axis=0))
        self.squared_sum.assign_add(tf.reduce_sum(y_true * weighted_y_true, axis=0))
        self.res.assign_add(
            tf.reduce_sum((y_true - y_pred) ** 2 * sample_weight, axis=0)
        )
        self.count.assign_add(tf.reduce_sum(sample_weight, axis=0))
        self.num_samples.assign_add(tf.size(y_true))

    def result(self) -> tf.Tensor:
        mean = self.sum / self.count
        total = self.squared_sum - self.sum * mean
        raw_scores = 1 - (self.res / total)
        raw_scores = tf.where(tf.math.is_inf(raw_scores), 0.0, raw_scores)

        if self.multioutput == "raw_values":
            r2_score = raw_scores
        elif self.multioutput == "uniform_average":
            r2_score = tf.reduce_mean(raw_scores)
        else:
            raise RuntimeError("The multioutput attribute is not Valid!!")

        if self.num_regressors < 0:
            raise ValueError("num_regressors parameter should be greater than or equal to zero")

        if self.num_regressors == 0:
            n = tf.cast(self.num_samples, dtype=tf.float32)
            p = tf.cast(self.num_regressors, dtype=tf.float32)

            num = tf.multiply(tf.subtract(1.0, r2_score), tf.subtract(n, 1.0))
            den = tf.subtract(tf.subtract(n, p), 1.0)
            r2_score = tf.subtract(1.0, tf.divide(num, den))

        return r2_score

    def reset_state(self) -> None:
        # The state of the metric will be reset at the start of each epoch.
        K.batch_set_value([(v, np.zeros(v.shape)) for v in self.variables])

    def get_config(self):
        config = {
            "y_shape": self.y_shape,
            "multioutput": self.multioutput,
        }
        base_config = super().get_config()
        return {**base_config, **config}




class RootMeanSquaredError(tf.keras.metrics.Metric):
    def __init__(
        self,
        uncertainty: bool = False,
        name: str = "root_mean_squared_error",
        dtype: AcceptableDTypes = None,
        y_shape: Tuple[int, ...] = (),
        **kwargs,
    ):
        super(RootMeanSquaredError, self).__init__(name, dtype=dtype)

        self.uncertainty = uncertainty
        self.y_shape = y_shape

        self.squared_sum = self.add_weight(
            name="squared_sum", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.count = self.add_weight(
            name="count", shape=y_shape, initializer="zeros", dtype=dtype
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, self._dtype)
        y_pred = tf.cast(y_pred, self._dtype)

        if self.uncertainty:
           y_pred = y_pred[:, ::2]

        if sample_weight is None:
            sample_weight = 1
        sample_weight = tf.cast(sample_weight, dtype=self._dtype)
        sample_weight = weights_broadcast_ops.broadcast_weights(
            weights=sample_weight, values=y_true
        )

        self.squared_sum.assign_add(
            tf.reduce_sum((y_true - y_pred) ** 2 * sample_weight, axis=0)
        )
        self.count.assign_add(tf.reduce_sum(sample_weight, axis=0))

    
    def result(self) -> tf.Tensor:
        return tf.sqrt(tf.math.divide_no_nan(self.squared_sum, self.count))

    def reset_state(self) -> None:
        # The state of the metric will be reset at the start of each epoch.
        K.batch_set_value([(v, np.zeros(v.shape)) for v in self.variables])

    def get_config(self):
        config = {
            "y_shape": self.y_shape,
        }
        base_config = super().get_config()
        return {**base_config, **config}



class MeanAbsoluteError(tf.keras.metrics.Metric):  
    def __init__(
        self,
        uncertainty: bool = False,
        name: str = "mean_absolte_error",
        dtype: AcceptableDTypes = None,
        y_shape: Tuple[int, ...] = (),
        **kwargs,
    ):
        super(MeanAbsoluteError, self).__init__(name, dtype=dtype)

        self.uncertainty = uncertainty
        self.y_shape = y_shape

        self.absolte_sum = self.add_weight(
            name="absolte_sum", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.count = self.add_weight(
            name="count", shape=y_shape, initializer="zeros", dtype=dtype
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, self._dtype)
        y_pred = tf.cast(y_pred, self._dtype)

        if self.uncertainty:
            y_pred = y_pred[:, ::2]

        if sample_weight is None:
            sample_weight = 1
        sample_weight = tf.cast(sample_weight, dtype=self._dtype)
        sample_weight = weights_broadcast_ops.broadcast_weights(
            weights=sample_weight, values=y_true
        )

        self.absolte_sum.assign_add(
            tf.reduce_sum(tf.abs(y_true - y_pred) * sample_weight, axis=0)
        )
        self.count.assign_add(tf.reduce_sum(sample_weight, axis=0))

    
    def result(self) -> tf.Tensor:
        return tf.math.divide_no_nan(self.absolte_sum, self.count)

    def reset_state(self) -> None:
        # The state of the metric will be reset at the start of each epoch.
        K.batch_set_value([(v, np.zeros(v.shape)) for v in self.variables])

    def get_config(self):
        config = {
            "y_shape": self.y_shape,
        }
        base_config = super().get_config()
        return {**base_config, **config}


class Sensitivity(tf.keras.metrics.Metric):  
    def __init__(
        self,
        main_prediction=None,
        uncertainty: bool = False,
        name: str = "sensitivity",
        dtype: AcceptableDTypes = None,
        y_shape: Tuple[int, ...] = (),
        **kwargs,
    ):
        super(Sensitivity, self).__init__(name, dtype=dtype)
        self.main_prediction = main_prediction
        if self.main_prediction is not None:
            self.main_prediction = tf.cast(self.main_prediction[:,::2], self._dtype)

        self.uncertainty = uncertainty
        self.y_shape = y_shape

        self.sum_change_percentage = self.add_weight(
            name="sum_change_percentage", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.count = self.add_weight(
            name="count", shape=y_shape, initializer="zeros", dtype=dtype
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, self._dtype)
        y_pred = tf.cast(y_pred, self._dtype)

        if self.uncertainty:
            y_pred = y_pred[:, ::2]
        
        if self.main_prediction is None:
            main_prediction = y_true
        else:
            main_prediction = self.main_prediction

        if sample_weight is None:
            sample_weight = 1
        sample_weight = tf.cast(sample_weight, dtype=self._dtype)
        sample_weight = weights_broadcast_ops.broadcast_weights(
            weights=sample_weight, values=y_true
        )

        try:
            change_percentage = 100.0 * tf.abs((main_prediction - y_pred) / (main_prediction + 1e-8)) * sample_weight
        except:
            change_percentage = 100.0 * tf.abs((y_true - y_pred) / (y_true + 1e-8))


        self.sum_change_percentage.assign_add(
            tf.reduce_sum(change_percentage, axis=0)
        )
        self.count.assign_add(tf.reduce_sum(sample_weight, axis=0))

    
    def result(self) -> tf.Tensor:
        return tf.math.divide_no_nan(self.sum_change_percentage, self.count)

    def reset_state(self) -> None:
        # The state of the metric will be reset at the start of each epoch.
        K.batch_set_value([(v, np.zeros(v.shape)) for v in self.variables])

    def get_config(self):
        config = {
            "y_shape": self.y_shape,
        }
        base_config = super().get_config()
        return {**base_config, **config}




class STD(tf.keras.metrics.Metric):  
    def __init__(
        self,
        uncertainty: bool = False,
        name: str = "std",
        dtype: AcceptableDTypes = None,
        y_shape: Tuple[int, ...] = (),
        **kwargs,
    ):
        super(STD, self).__init__(name, dtype=dtype)
        self.uncertainty = uncertainty
        self.y_shape = y_shape

        self.sum_std = self.add_weight(
            name="sum_std", shape=y_shape, initializer="zeros", dtype=dtype
        )
        self.count = self.add_weight(
            name="count", shape=y_shape, initializer="zeros", dtype=dtype
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, self._dtype)
        y_pred = tf.cast(y_pred, self._dtype)

        if self.uncertainty:
            y_pred = y_pred[:, 1::2]

        if sample_weight is None:
            sample_weight = 1
        sample_weight = tf.cast(sample_weight, dtype=self._dtype)
        sample_weight = weights_broadcast_ops.broadcast_weights(
            weights=sample_weight, values=y_true
        )

        self.sum_std.assign_add(
            tf.reduce_sum(y_pred * sample_weight, axis=0)
        )
        self.count.assign_add(tf.reduce_sum(sample_weight, axis=0))

    
    def result(self) -> tf.Tensor:
        return tf.math.divide_no_nan(self.sum_std, self.count)

    def reset_state(self) -> None:
        # The state of the metric will be reset at the start of each epoch.
        K.batch_set_value([(v, np.zeros(v.shape)) for v in self.variables])

    def get_config(self):
        config = {
            "y_shape": self.y_shape,
        }
        base_config = super().get_config()
        return {**base_config, **config}

In [ ]:
class GaussLoss(tf.keras.losses.Loss):
    """Negative log likelihood of y_true, with the likelihood defined by a normal distribution."""
    def __init__(self, name="loss"):
        super(GaussLoss, self).__init__(name=name)

    def call(self, y_true, y_pred):
        means = y_pred[:, ::2]
        # We predict the log of the standard deviation, so exponentiate the prediction here
        stds = tf.exp(y_pred[:, 1::2])
        variances = stds * stds


        log_p = (-tf.math.log(tf.math.sqrt(2 * math.pi * variances) + 1e-8)
                 -(y_true - means) * (y_true - means) / (2 * variances))
        return tf.reduce_mean(-log_p, axis=-1)  # Note the `axis=-1`

In [ ]:
epsilon = 5.0
def MAEM(y_true, y_pred):
    difference = tf.abs(y_true - y_pred) - epsilon
    return tf.nn.relu(difference)

In [ ]:
# Deep Neural Network
def create_model(in_features, o_features, n_h_layers=6, drop_rate=0.25, h_units=128):
    inputs = tf.keras.layers.Input(shape=(in_features,))
    x = inputs

    for _ in range(n_h_layers):
        x = tf.keras.layers.Dense(h_units, activation="relu")(x)
        x = tf.keras.layers.Dropout(drop_rate)(x)

    x = tf.keras.layers.Dense(o_features)(x)
    outputs = tf.keras.layers.ReLU(max_value=100.0)(x)

    model = tf.keras.models.Model(inputs, outputs)
    
    return model

In [ ]:
def trainer(model, train_data, test_data, initial_weights_path, epochs, uncertainty=False, main_prediction=None):
    model.load_weights(initial_weights_path)

    if uncertainty:
        optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, clipvalue=1.0)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)

    sensitivity = Sensitivity(main_prediction=main_prediction, uncertainty=uncertainty, y_shape=train_data[1].shape[-1], name='sensitivity')
    r2score = RSquare(uncertainty=uncertainty, y_shape=train_data[1].shape[-1], name='r_square')
    rsme = RootMeanSquaredError(uncertainty=uncertainty, y_shape=train_data[1].shape[-1], name='RMSE')
    mae = MeanAbsoluteError(uncertainty=uncertainty, y_shape=train_data[1].shape[-1], name='MAE')
    
    metrics = [sensitivity, r2score, rsme, mae]
    if uncertainty:
        metrics.append(STD(uncertainty=uncertainty, y_shape=train_data[1].shape[-1], name='STD'))


    if uncertainty:
        loss = GaussLoss()
    else:
        loss = MAEM 
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics,
                  )
    
    callbacks = [tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0),
                 BestModelWeights(),
                 ShowProgress(epochs),
                 ]

    history = model.fit(train_data[0], train_data[1],
                        epochs=epochs,
                        batch_size=len(train_data[0]),
                        validation_data=test_data,
                        callbacks=callbacks,
                        verbose=0).history
    
    #best_index = np.argmin(history['val_loss'])
    best_index = np.argmin(history['val_sensitivity'])

    if uncertainty:
        return np.array([history['MAE'][best_index],
                        history['sensitivity'][best_index],
                        history['RMSE'][best_index],
                        history['r_square'][best_index],
                        history['val_MAE'][best_index],
                        history['val_sensitivity'][best_index],
                        history['val_RMSE'][best_index],
                        history['val_r_square'][best_index],
                        history['STD'][best_index],
                        history['val_STD'][best_index],
                         ])
    else:
        return np.array([history['MAE'][best_index],
                        history['sensitivity'][best_index],
                        history['RMSE'][best_index],
                        history['r_square'][best_index],
                        history['val_MAE'][best_index],
                        history['val_sensitivity'][best_index],
                        history['val_RMSE'][best_index],
                        history['val_r_square'][best_index],
                         ])

In [ ]:
!rm -rf Result

In [ ]:
os.mkdir("Result")

In [ ]:
EPOCH = 1000
n_splits = 5
n_repeats = 3
uncertainty = False
initial_weights = "./Result/initial_weights.h5"
n_output = 1

if uncertainty:
    n_output = 2 * n_output

kf = KFold(n_splits=n_splits)
X, Y = data_[:, :11], data_[:, 11:]

model = create_model(X.shape[-1], n_output)
model.save_weights(initial_weights)

if uncertainty:
    Result_No_Dropping = np.zeros((n_splits, 10, n_repeats))
else:
    Result_No_Dropping = np.zeros((n_splits, 8, n_repeats))
for counter, (train_index, test_index) in enumerate(kf.split(X)):
    if uncertainty:
        print(58 * "*", f"Fold {counter + 1} Started!", 59 * "*")
    else:
        print(49 * "*", f"Fold {counter + 1} Started!", 50 * "*")
    
    train_x, train_y = X[train_index], Y[train_index]
    test_x, test_y = X[test_index], Y[test_index]
    

    ############################ With target normalization ###############################
    scaler_x = MinMaxScaler()

    scaler_x.fit(X)
    train_x = scaler_x.transform(train_x)
    test_x = scaler_x.transform(test_x)
    #######################################################################################



    train_y_EE = train_y[:, 0][..., None]
    test_y_EE = test_y[:, 0][..., None]

    print(f"Fold {counter + 1} , Without Dropping\n")
    main_prediction = 0
    for counter1 in range(n_repeats):
        print(f"Repeat : {counter1 + 1}")
        model = create_model(train_x.shape[-1], n_output)
        best_result_metrics = trainer(model, (train_x, train_y_EE), (test_x, test_y_EE), initial_weights, EPOCH, uncertainty)
        #main_prediction += model(test_x)[:,::2]
        main_prediction += model(test_x)

        Result_No_Dropping[counter, :, counter1] = best_result_metrics
    main_prediction = main_prediction / n_repeats

    if uncertainty:
        print(f"Train [ MAE={round(Result_No_Dropping[counter,0,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,2,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,3,:].mean(), 4)}, STD={round(Result_No_Dropping[counter,8,:].mean(), 4)} ], Test [ MAE={round(Result_No_Dropping[counter,4,:].mean(), 4)}, Sensivity={round(Result_No_Dropping[counter,5,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,6,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,7,:].mean(), 4)}, STD={round(Result_No_Dropping[counter,9,:].mean(), 4)} ]")
        print(143 * "*")
    else:
        print(f"Train [ MAE={round(Result_No_Dropping[counter,0,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,2,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,3,:].mean(), 4)} ], Test [ MAE={round(Result_No_Dropping[counter,4,:].mean(), 4)}, Sensivity={round(Result_No_Dropping[counter,5,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,6,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,7,:].mean(), 4)} ]")
        print(114 * "*")

In [ ]:
Result_No_Dropping[:,4,:].mean()

In [ ]:
with open('./Result/Result_No_Dropping.npy', 'wb') as f:
     np.save(f, Result_No_Dropping)

In [ ]:
!zip -r metrics.zip /content/Result

In [ ]:
try:
    from google.colab import files
    files.download('metrics.zip')
except:
    pass

In [ ]:
!rm -rf Result

In [ ]:
os.mkdir("Result")

In [ ]:
EPOCH = 1000
n_splits = 5
n_repeats = 3
uncertainty = False
initial_weights = "./Result/initial_weights.h5"
n_output = 1

if uncertainty:
    n_output = 2 * n_output

kf = KFold(n_splits=n_splits)
X, Y = data_[:, :11], data_[:, 11:]

model = create_model(X.shape[-1], n_output)
model.save_weights(initial_weights)

if uncertainty:
    Result_No_Dropping = np.zeros((n_splits, 10, n_repeats))
    Results_Dropping_EE = np.zeros((n_splits, X.shape[-1], 10, n_repeats))
else:
    Result_No_Dropping = np.zeros((n_splits, 8, n_repeats))
    Results_Dropping_EE = np.zeros((n_splits, X.shape[-1], 8, n_repeats))
for counter, (train_index, test_index) in enumerate(kf.split(X)):
    if uncertainty:
        print(58 * "*", f"Fold {counter + 1} Started!", 59 * "*")
    else:
        print(49 * "*", f"Fold {counter + 1} Started!", 50 * "*")
    
    train_x, train_y = X[train_index], Y[train_index]
    test_x, test_y = X[test_index], Y[test_index]
    

    ############################ With target normalization ###############################
    scaler_x = MinMaxScaler()

    scaler_x.fit(X)
    train_x = scaler_x.transform(train_x)
    test_x = scaler_x.transform(test_x)
    #######################################################################################



    train_y_EE = train_y[:, 0][..., None]
    test_y_EE = test_y[:, 0][..., None]

    print(f"Fold {counter + 1} , Without Dropping\n")
    main_prediction = 0
    for counter1 in range(n_repeats):
        print(f"Repeat : {counter1}")
        model = create_model(train_x.shape[-1], n_output)
        best_result_metrics = trainer(model, (train_x, train_y_EE), (test_x, test_y_EE), initial_weights, EPOCH, uncertainty)
        #main_prediction += model(test_x)[:,::2]
        main_prediction += model(test_x)

        Result_No_Dropping[counter, :, counter1] = best_result_metrics
    main_prediction = main_prediction / n_repeats

    if uncertainty:
        print(f"Train [ MAE={round(Result_No_Dropping[counter,0,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,2,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,3,:].mean(), 4)}, STD={round(Result_No_Dropping[counter,8,:].mean(), 4)} ], Test [ MAE={round(Result_No_Dropping[counter,4,:].mean(), 4)}, Sensivity={round(Result_No_Dropping[counter,5,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,6,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,7,:].mean(), 4)}, STD={round(Result_No_Dropping[counter,9,:].mean(), 4)} ]")
        print(143 * "*")
    else:
        print(f"Train [ MAE={round(Result_No_Dropping[counter,0,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,2,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,3,:].mean(), 4)} ], Test [ MAE={round(Result_No_Dropping[counter,4,:].mean(), 4)}, Sensivity={round(Result_No_Dropping[counter,5,:].mean(), 4)}, RMSE={round(Result_No_Dropping[counter,6,:].mean(), 4)}, R2={round(Result_No_Dropping[counter,7,:].mean(), 4)} ]")
        print(114 * "*")

    for counter1 in range(X.shape[1]):
        start = time.time()
        print(f"Fold :  {counter + 1},  Dropping Feature : {counter1 + 1}\n")
        for counter2 in range(n_repeats):
            print(f"Repeat : {counter2}")
            croped_train_x, croped_test_x = train_x.copy(), test_x.copy()

            croped_train_x[:, counter1] = -1
            croped_test_x[:, counter1] = -1
            
            
            model = create_model(croped_train_x.shape[-1], n_output)
            best_result_metrics = trainer(model, (croped_train_x, train_y_EE), (croped_test_x, test_y_EE), initial_weights, EPOCH, uncertainty, main_prediction)

            Results_Dropping_EE[counter, counter1, :, counter2] = best_result_metrics

        if uncertainty:
            print(f"Train [ MAE={round(Results_Dropping_EE[counter,counter1,0,:].mean(), 4)}, RMSE={round(Results_Dropping_EE[counter,counter1,2,:].mean(), 4)}, R2={round(Results_Dropping_EE[counter,counter1,3,:].mean(), 4)}, STD={round(Results_Dropping_EE[counter,counter1,8,:].mean(), 4)} ], Test [ MAE={round(Results_Dropping_EE[counter,counter1,4,:].mean(), 4)}, Sensivity={round(Results_Dropping_EE[counter,counter1,5,:].mean(), 4)}, RMSE={round(Results_Dropping_EE[counter,counter1,6,:].mean(), 4)}, R2={round(Results_Dropping_EE[counter,counter1,7,:].mean(), 4)}, STD={round(Results_Dropping_EE[counter,counter1,9,:].mean(), 4)} ]")
            print(f"Time : {round(time.time() - start, 4)}")
            print(143 * "*")
        else:
            print(f"Train [ MAE={round(Results_Dropping_EE[counter,counter1,0,:].mean(), 4)}, RMSE={round(Results_Dropping_EE[counter,counter1,2,:].mean(), 4)}, R2={round(Results_Dropping_EE[counter,counter1,3,:].mean(), 4)} ], Test [ MAE={round(Results_Dropping_EE[counter,counter1,4,:].mean(), 4)}, Sensivity={round(Results_Dropping_EE[counter,counter1,5,:].mean(), 4)}, RMSE={round(Results_Dropping_EE[counter,counter1,6,:].mean(), 4)}, R2={round(Results_Dropping_EE[counter,counter1,7,:].mean(), 4)} ]")
            print(f"Time : {round(time.time() - start, 4)}")
            print(114 * "*")

In [ ]:
with open('./Result/Results_Dropping.npy', 'wb') as f:
     np.save(f, Results_Dropping_EE)     

with open('./Result/Result_No_Dropping.npy', 'wb') as f:
     np.save(f, Result_No_Dropping)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(Results_Dropping_EE[:,:,5,:].mean(axis=0).mean(axis=-1))
plt.plot([Result_No_Dropping.mean(axis=0).mean(axis=-1)[5]] * 11)
plt.xticks(range(11), list(data.columns)[:11], rotation=90)

plt.show()

In [ ]:
!zip -r metrics.zip /content/Result

In [ ]:
try:
    from google.colab import files
    files.download('metrics.zip')
except:
    pass

In [ ]:
plt.plot(Results_Dropping_EE[:,:,5,:].mean(axis=-1).mean(axis=0))
plt.show()

In [ ]:
with open('./Result/Results_Dropping.npy', 'wb') as f:
     np.save(f, Results_Dropping_EE)   

In [ ]:
data_design = pd.read_excel(os.path.join(base_path, "Data_Design.xlsx"))
data_design = data_design.to_numpy().astype(np.float32)

X_design = data_design[:, 1:12]
Y_design = data_design[:, 12]

In [ ]:
X_design = scaler_x.transform(X_design)

In [ ]:
x_actual = X_design[[0, 3, 6, 9], 6]
y_actual = Y_design[[0, 3, 6, 9]]

print(x_actual)
print(y_actual)

In [ ]:
n_data = 100

buff = np.tile(X_design[0][None, ...], (n_data, 1))
for counter, hlp in enumerate(np.linspace(0, 1, n_data)):
    buff[counter, 7] = hlp

In [ ]:
prediction = model.predict(buff)

In [ ]:
def visualize_prediction(prediction, actual, filename=None, bgcolor="#ffffff"):
    
    figure = plt.figure()
    
    means, stds = prediction[:,0], prediction[:,1]
    
    plt.plot(np.linspace(0, 1, n_data), means)
    plt.fill_between(np.linspace(0, 1, n_data), means-stds, means+stds, color='blue', alpha=0.1)
    plt.plot(actual[0], actual[1], "ro")
    plt.ylim([0, 100])
    
    if filename is not None:
        plt.savefig(filename, bbox_inches="tight", dpi=200)

    plt.show()

In [ ]:
visualize_prediction(prediction, (x_actual, y_actual))